# Benchmark ReQA
- Input: `06_calculate_distance.parquet`

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from utils import encode_in_batch

import tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub

2023-08-29 18:48:00.792496: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-29 18:48:00.816044: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-29 18:48:01.103572: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
context_data = pd.read_parquet('data/06_calculate_distance.parquet')
context_data

,id,context,th_aug,th_fasttext_aug,th_llm_gec_aug,th_llm_paraphrase_aug,th_qcpg_0.2_aug,th_qcpg_0.2_llm_gec_aug,th_qcpg_0.5_aug,th_qcpg_0.5_llm_gec_aug,...,dis_llm_paraphrase_aug,dis_qcpg_0.2_aug,dis_qcpg_0.2_llm_gec_aug,dis_qcpg_0.5_aug,dis_qcpg_0.5_llm_gec_aug,dis_qcpg_0.8_aug,dis_qcpg_0.8_llm_gec_aug,dis_wordnet_aug,context_id,source
0,9ce376f4446e5b43e55d6f545c1adec7975a51d9085156...,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,Phatthira Saruritphong Phokin (เกิด 3 ธันวาคม ...,พัทธ์ธีดาเล ศรุติพงศ์โภคิน (เกิด 3 21มกราคม พ....,Phatthira Saruritphong Phokin (เกิด 3 ธันวาคม ...,Phatthira Saruritphong Phokin ซึ่งเกิดเมื่อวัน...,Phatthira Saruuritt Phokin (เกิดเมื่อวันที่ 3 ...,Phatthira Saruutfokin (เกิดเมื่อวันที่ 3 ธันวา...,ชื่อของ Phatthira Saruuritt Phokin (เกิดในโรงเ...,Phatthira Saruutfokin (เกิดเมื่อวันที่ 3 ธันวา...,...,0.184912,0.436465,0.437679,0.501482,0.370968,0.450085,0.369090,0.022316,9ce376f4446e5b43e55d6f545c1adec7975a51d9085156...,iapp
1,54421ea8bf2c7778b1d27a9e878c31aee3361455e767f0...,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,คลีโอพัตรา 7 ฟิโลพาเธอร์ (กรีก: κλεοπάτραθεάφι...,คลีโอพัตราที่ 58 Zafina ทริงเงิพาเธอร์ (กรีก :...,Cleopatra VII Philopator (กรีก: κλεοπάτραθεάφι...,Cleopatra VII Philopator หรือที่รู้จักกันในชื่...,"- 69 - 69 - โต๊ะ (Laug พ่อของเธอ A Wolf Mi, 12...",Cleopatra VII หรือที่รู้จักกันในชื่อ Cleopatra...,ดังนั้นครอบครัวกรีกคนสุดท้ายของราชวงศ์สตากา-เม...,Cleopatra VII หรือที่รู้จักกันในชื่อคลีโอพัตรา...,...,0.141594,0.292852,0.208292,0.220980,0.210469,0.274992,0.226844,0.018455,54421ea8bf2c7778b1d27a9e878c31aee3361455e767f0...,iapp
2,287fa29b965a1ba84011bb12ffafb5194984a955d851eb...,ออสเตรเลีย (อังกฤษ: Australia) หรือชื่อทางการค...,ออสเตรเลียหรือชื่อทางการคือกลุ่มออสเตรเลีย (เค...,ออสเตรเลีย6 (อังกฤษ : Australia ) บางทีชื่อทาง...,ออสเตรเลียซึ่งเป็นที่รู้จักอย่างเป็นทางการในฐา...,ออสเตรเลียซึ่งเป็นที่รู้จักอย่างเป็นทางการในฐา...,"กลุ่มออสเตรเลีย (ออสเตรเลีย), Tae-Samanthin แล...",ออสเตรเลียซึ่งเรียกว่าออสเตรเลียอย่างเป็นทางกา...,"กลุ่มออสเตรเลีย (ออสเตรเลีย), Tae-Samanthin แล...",กลุ่มออสเตรเลียหรือที่รู้จักกันในชื่อกลุ่มออสเ...,...,0.061920,0.149705,0.146146,0.117441,0.146885,0.128104,0.151757,0.024578,287fa29b965a1ba84011bb12ffafb5194984a955d851eb...,iapp
3,35adee7e78d0ae214238db68c9b524e01b1997ee7bf1ad...,วันสันติภาพไทย คือ วันที่ 16 สิงหาคม ของทุกปี ...,วันสันติภาพไทยเป็นวันที่ 16 สิงหาคมทุกปีซึ่งเป...,เมื่ออนุสนธิการ์ตูน2ภาษา เรียก วันที่ YIMSOO ส...,วันแห่งความสงบสุขของไทยมีการเฉลิมฉลองวันที่ 16...,วันสันติภาพไทยมีการเฉลิมฉลองเป็นประจำทุกปีในวั...,วันสันติภาพไทยเป็นวันที่ 16 ของ AUGUST 1945 แล...,วันแห่งความสงบสุขของไทยมีการเฉลิมฉลองในวันที่ ...,วันสันติภาพไทยเป็นวันที่ 16 ของ AUGUST 1945 แล...,วันแห่งความสงบสุขของไทยมีการเฉลิมฉลองในวันที่ ...,...,0.136226,0.256062,0.181397,0.295456,0.207665,0.227967,0.247443,0.010985,35adee7e78d0ae214238db68c9b524e01b1997ee7bf1ad...,iapp
4,e4952f44ce25aa622976eb9d382ae03cfa38d3d081f0df...,นิวซีแลนด์ (อังกฤษ: New Zealand มาวรี: Aotearo...,นิวซีแลนด์ (ภาษาอังกฤษ: นิวซีแลนด์: Aotearoa ห...,นิวซีแลนด์ (อังกฤษ : New Zealand มาวอัลแฟ : Ao...,นิวซีแลนด์ (ภาษาอังกฤษ: นิวซีแลนด์ เมารี: Aote...,นิวซีแลนด์หรือที่รู้จักกันในชื่อ Aotearoa หรือ...,นิวซีแลนด์ (Aotearoa) เป็นดินแดนที่มีเมฆสีขาวอ...,"นิวซีแลนด์ (Aotearoa, นิวซีแลนด์, ดินแดนแห่งเม...",นิวซีแลนด์ (Aotearoa: Aotearoa ประเทศฟินแลนด์ป...,"นิวซีแลนด์ (Aotearoa, นิวซีแลนด์และสาธารณรัฐอั...",...,0.092218,0.196770,0.132868,0.152546,0.152899,0.167104,0.194042,0.014894,e4952f44ce25aa622976eb9d382ae03cfa38d3d081f0df...,iapp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7838,5f74195cacdd4383213ee1ad9cc2e41497f0b92a499572...,สีดำ หมายถึง สีแห่งศอของพระศิวะที่ดื่มยาพิษเพื...,สีดำหมายถึงสีของพระอิศวรที่ดื่มเพื่อปกป้องมนุษ...,สีดำ หมายถึง สีแห่งศอของพระศิวะที่ดื่มยาพิษเพื...,สีดำเป็นสัญลักษณ์ของอะไรในบริบทของเรื่องราวของ...,สีดำเป็นสีที่เกี่ยวข้องกับพระอิศวรซึ่งเป็นการแ...,สีดำเป็นสีของพระอิศวรคนที่ดื่มเพื่อปกป้องมนุษย...,สีดำคืออะไรเมื่อเผชิญกับเรื่องราวของ SIV เก

In [3]:
questions_data = pd.read_parquet('../questions/data/06_calculate_distance.parquet')
questions_data

,id,question,context,answers,source,th_aug,th_fasttext_aug,th_llm_gec_aug,th_llm_paraphrase_aug,th_ltw2v_aug,...,dis_ltw2v_aug,dis_qcpg_0.2_aug,dis_qcpg_0.2_llm_gec_aug,dis_qcpg_0.5_aug,dis_qcpg_0.5_llm_gec_aug,dis_qcpg_0.8_aug,dis_qcpg_0.8_llm_gec_aug,dis_thai2fit_aug,dis_thai2trans_aug,dis_wordnet_aug
0,82c589fda5d3007be968f303e04a9ac58953299a64a5be...,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['3 ธันวาคม พ.ศ. 2533'], 'answer_star...",iapp,Phatthira Sarutpong Phokin วันเดือนปีเกิดคืออะไร?,พัณณ์ธีภ ศรุติวัฒน์2โภคิน เกิดวันที่ไหร่?,วันเดือนปีเกิดของ Phatthira Sarutpong Phokin ค...,Phatthira Sarutpong Phokin คุณช่วยจัดเตรียมวัน...,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำให้เกิดวันอาทิตย์เท...,...,0.379822,0.457964,0.442132,0.442132,0.664607,0.373971,0.391374,0.213284,0.000000,9.606189e-02
1,6483e513603bd34b6f23a5fbe9e1c1db194ce4f4365a69...,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['อร'], 'answer_start': [68], 'answer...",iapp,Phatthira Sarutpong Phokin คุณกำลังเล่นอะไรอยู่?,พัทธ์ธีรา ศิษฎพงศ์อรัช dcccxxxii ในPSPเล่นว่าอ...,คุณกำลังเล่นอะไร Phatthira Sarutpong Phokin?,Phatthira Sarutpong Phokin คุณกำลังเล่นเกมอะไร...,พัทธ์ธีรา ศรุติพงษ์โภคิน มีฃื่อเล่นว่าอะไร?,...,0.028428,0.512295,0.604564,0.552465,0.590820,0.583206,0.645776,0.232190,0.000000,0.000000e+00
2,09fc57198420c10d8cd67154a154cca37fdcde53ec894a...,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['นักแสดงหญิงชาวไทย'], 'answer_start'...",iapp,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,รพนธ์ธีคาต ศรุติวรรณ2โภคิน ทำอาชีพอะไร4?,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำอาชีพอะไร?,...,0.017806,0.408607,0.408607,0.346143,0.346143,0.328919,0.328919,0.150410,0.000000,0.000000e+00
3,7b2f336d3dbc29c6fb1edf7f4988f968ed0a1272fe8aff...,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['ประเทศนิวซีแลนด์'], 'answer_start':...",iapp,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศ...,พัทธ์ธีรา ตอัตถพงศ์ฯกุล1 จบการศึกษาจากภ.ต่างอะ...,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศใด?,Phatthira Sarutpong Phokin จากประเทศใดจากการศึ...,พัทธ์ธีรา ศรุติพงษ์โภคิน จบการศึกษาจากในประเทศ...,...,0.044393,0.600420,0.717404,0.606477,0.717404,0.433692,0.523367,0.155829,0.009204,1.820072e-01
4,50c66c986cdf19b60f36da1b89137a6ceb4f11aa9ed0e9...,บิดาของคลีโอพัตราเป็นใคร?,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,"{'text': ['ทอเลมีที่ 12 ออเลติส'], 'answer_sta...",iapp,พ่อของคลีโอพัตราคือใคร?,บิดาของมเนสโอพัตตกฺขหลายๆใคร?,พ่อของคลีโอพัตราคือใคร?,พ่อผู้ให้กำเนิดของคลีโอพัตราคือใคร?,มารดาของคลีโอพัตราเป็นใคร?,...,0.180941,0.238710,0.388979,0.151006,0.443043,0.097685,0.097685,0.150196,0.156331,8.621460e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16975,e780cc4812dd88f21ec7f0152905b15588d1a6faab66d2...,หนังสือการ์ตูนชานะ นักรบเนตรอัคคี มีกี่เล่ม?,นิยาย ชานะ นักรบเนตรอัคคี แต่งโดย ยาชิจิโร ทาค...,"{'text': ['22', '26'], 'answer_start': [267, 9...",tydiqa,หนังสือการ์ตูนนักรบดับเพลิงกี่คน?,หนังสือDIYการ์ตูนPokemonชานะ อุดมการณ์ถ.ทิพยเน...,นักรบดับเพลิงมีกี่คนในหนังสือการ์ตูน?,ในหนังสือการ์ตูนมีนักรบดับเพลิงกี่คน?,หนังสือการ์ตูนชาน่ะ นักรบเนตรอัคคี มีกี่เล่ม?,...,0.038205,0.459041,0.345795,0.330124,0.345795,0.363513,0.284700,0.063899,0.012518,1.700147e-01
16976,dd424979403f3909773a832c4d83818e564fe7bfeb4f38...,ไบโอช็อก อินฟินิต เปิดตัวครั้งแรกเมื่อไหร่?,ไบโอช็อก อินฟินิต (English: BioShock Infinite)...,"{'text': ['26 มีนาคม พ.ศ. 2556', '26 มีนาคม พ....",tydiqa,BioShock Infin เปิดตัวครั้งแรกเมื่อใด?,ไบโอมีช๊อค อร์เนชยมีฟิตี้07ต เปิดตัวครั้งแรกเม...,BioShock Infinite เปิดตัวครั้งแรกเมื่อใด?,การเปิดตัวครั้งแรกของ BioShock Infinite คือเมื...,ไบโอช็อค อินฟินิต เผยโฉมเป็นครั้งแรกเมื่อไร?,...,0.168088,0.609325,0.713147,0.475034,0.686884,0.482104,0.602809,0.097

In [4]:
import hashlib

# Sha256 
questions_data["context_id"] = questions_data["context"].apply(lambda x: hashlib.sha256(x.encode("utf-8")).hexdigest())
questions_data

,id,question,context,answers,source,th_aug,th_fasttext_aug,th_llm_gec_aug,th_llm_paraphrase_aug,th_ltw2v_aug,...,dis_qcpg_0.2_aug,dis_qcpg_0.2_llm_gec_aug,dis_qcpg_0.5_aug,dis_qcpg_0.5_llm_gec_aug,dis_qcpg_0.8_aug,dis_qcpg_0.8_llm_gec_aug,dis_thai2fit_aug,dis_thai2trans_aug,dis_wordnet_aug,context_id
0,82c589fda5d3007be968f303e04a9ac58953299a64a5be...,พัทธ์ธีรา ศรุติพงศ์โภคิน เกิดวันที่เท่าไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['3 ธันวาคม พ.ศ. 2533'], 'answer_star...",iapp,Phatthira Sarutpong Phokin วันเดือนปีเกิดคืออะไร?,พัณณ์ธีภ ศรุติวัฒน์2โภคิน เกิดวันที่ไหร่?,วันเดือนปีเกิดของ Phatthira Sarutpong Phokin ค...,Phatthira Sarutpong Phokin คุณช่วยจัดเตรียมวัน...,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำให้เกิดวันอาทิตย์เท...,...,0.457964,0.442132,0.442132,0.664607,0.373971,0.391374,0.213284,0.000000,9.606189e-02,9ce376f4446e5b43e55d6f545c1adec7975a51d9085156...
1,6483e513603bd34b6f23a5fbe9e1c1db194ce4f4365a69...,พัทธ์ธีรา ศรุติพงศ์โภคิน มีฃื่อเล่นว่าอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['อร'], 'answer_start': [68], 'answer...",iapp,Phatthira Sarutpong Phokin คุณกำลังเล่นอะไรอยู่?,พัทธ์ธีรา ศิษฎพงศ์อรัช dcccxxxii ในPSPเล่นว่าอ...,คุณกำลังเล่นอะไร Phatthira Sarutpong Phokin?,Phatthira Sarutpong Phokin คุณกำลังเล่นเกมอะไร...,พัทธ์ธีรา ศรุติพงษ์โภคิน มีฃื่อเล่นว่าอะไร?,...,0.512295,0.604564,0.552465,0.590820,0.583206,0.645776,0.232190,0.000000,0.000000e+00,9ce376f4446e5b43e55d6f545c1adec7975a51d9085156...
2,09fc57198420c10d8cd67154a154cca37fdcde53ec894a...,พัทธ์ธีรา ศรุติพงศ์โภคิน ทำอาชีพอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['นักแสดงหญิงชาวไทย'], 'answer_start'...",iapp,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,รพนธ์ธีคาต ศรุติวรรณ2โภคิน ทำอาชีพอะไร4?,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,อาชีพของ Phatthira Teerathiyapong Phokin คืออะไร?,พัทธ์ธีรา ศรุติพงษ์โภคิน ทำอาชีพอะไร?,...,0.408607,0.408607,0.346143,0.346143,0.328919,0.328919,0.150410,0.000000,0.000000e+00,9ce376f4446e5b43e55d6f545c1adec7975a51d9085156...
3,7b2f336d3dbc29c6fb1edf7f4988f968ed0a1272fe8aff...,พัทธ์ธีรา ศรุติพงศ์โภคิน จบการศึกษาจากประเทศอะไร?,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,"{'text': ['ประเทศนิวซีแลนด์'], 'answer_start':...",iapp,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศ...,พัทธ์ธีรา ตอัตถพงศ์ฯกุล1 จบการศึกษาจากภ.ต่างอะ...,Phatthira Sarutpong Phokin จบการศึกษาจากประเทศใด?,Phatthira Sarutpong Phokin จากประเทศใดจากการศึ...,พัทธ์ธีรา ศรุติพงษ์โภคิน จบการศึกษาจากในประเทศ...,...,0.600420,0.717404,0.606477,0.717404,0.433692,0.523367,0.155829,0.009204,1.820072e-01,9ce376f4446e5b43e55d6f545c1adec7975a51d9085156...
4,50c66c986cdf19b60f36da1b89137a6ceb4f11aa9ed0e9...,บิดาของคลีโอพัตราเป็นใคร?,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,"{'text': ['ทอเลมีที่ 12 ออเลติส'], 'answer_sta...",iapp,พ่อของคลีโอพัตราคือใคร?,บิดาของมเนสโอพัตตกฺขหลายๆใคร?,พ่อของคลีโอพัตราคือใคร?,พ่อผู้ให้กำเนิดของคลีโอพัตราคือใคร?,มารดาของคลีโอพัตราเป็นใคร?,...,0.238710,0.388979,0.151006,0.443043,0.097685,0.097685,0.150196,0.156331,8.621460e-02,54421ea8bf2c7778b1d27a9e878c31aee3361455e767f0...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16975,e780cc4812dd88f21ec7f0152905b15588d1a6faab66d2...,หนังสือการ์ตูนชานะ นักรบเนตรอัคคี มีกี่เล่ม?,นิยาย ชานะ นักรบเนตรอัคคี แต่งโดย ยาชิจิโร ทาค...,"{'text': ['22', '26'], 'answer_start': [267, 9...",tydiqa,หนังสือการ์ตูนนักรบดับเพลิงกี่คน?,หนังสือDIYการ์ตูนPokemonชานะ อุดมการณ์ถ.ทิพยเน...,นักรบดับเพลิงมีกี่คนในหนังสือการ์ตูน?,ในหนังสือการ์ตูนมีนักรบดับเพลิงกี่คน?,หนังสือการ์ตูนชาน่ะ นักรบเนตรอัคคี มีกี่เล่ม?,...,0.459041,0.345795,0.330124,0.345795,0.363513,0.284700,0.063899,0.012518,1.700147e-01,7f0b2774fd75cda7dd6e9e1c0722052099b18d83f6fd6e...
16976,dd424979403f3909773a832c4d83818e564fe7bfeb4f38...,ไบโอช็อก อินฟินิต เปิดตัวครั้งแรกเมื่อไหร่?,ไบโอช็อก อินฟินิต (English: BioShock Infinite)...,"{'text': ['26 มีนาคม พ.ศ. 2556', '26 มีนาคม พ....",tydiqa,BioShock Infin เปิดตัวครั้งแรกเมื่อใด?,ไบโอมีช๊อค อร์เน

In [5]:
# Assert that all unique ids match between questions and contexts
# If it does not match, check if context is missing

try:
    assert len(set(context_data["id"].unique()) - set(questions_data["context_id"].unique())) == 0
except AssertionError:
    print("Contexts missing in questions data")
    for i in set(context_data["id"].unique()) - set(questions_data["context_id"].unique()):
        print(context_data[context_data["id"] == i]["context"].values[1])
        print("")

In [6]:
# Add None so that the first benchmark only consists of the original questions
all_augment_cols = [None] + [col for col in context_data.columns if col.startswith('th_')]
all_augment_cols

[None,
 'th_aug',
 'th_fasttext_aug',
 'th_llm_gec_aug',
 'th_llm_paraphrase_aug',
 'th_qcpg_0.2_aug',
 'th_qcpg_0.2_llm_gec_aug',
 'th_qcpg_0.5_aug',
 'th_qcpg_0.5_llm_gec_aug',
 'th_qcpg_0.8_aug',
 'th_qcpg_0.8_llm_gec_aug',
 'th_wordnet_aug']

In [7]:
model_embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual/3")

2023-08-29 18:48:05.846726: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-29 18:48:05.860745: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [8]:
def evaluate(
    question_id, question_all, context_id, context_all, mrr_rank=10, status=True
):
    top_1 = 0
    top_5 = 0
    top_10 = 0
    mrr_score = 0
    context_id = np.array(context_id)
    sim_score = np.inner(question_all, context_all)

    if status == True:
        status_bar = enumerate(tqdm(sim_score))
    else:
        status_bar = enumerate(sim_score)
    for idx, sim in status_bar:
        index = np.argsort(sim)[::-1]
        index_edit = [context_id[x] for x in index]
        try:
            idx_search = list(index_edit).index(question_id[idx])
        except:
            idx_search = 999999
        if idx_search == 0:
            top_1 += 1
            top_5 += 1
            top_10 += 1
        elif idx_search < 5:
            top_5 += 1
            top_10 += 1
            
            # # Debugging
            # print(idx_search)
            # print(index_edit)
            # print(question_id[idx])
            # print(f"Question: {questions_data[questions_data['context_id'] == question_id[idx]]['question'].values[0]}")
            # print(f"Context: {context_data[context_data['id'] == question_id[idx]]['context'].values[0]}")
            # print(f"GT Retrieved Context: {context_data[context_data['id'] == index_edit[idx_search]]['context'].values[0]}")
            # # Print all retrieved contexts top-15
            # print("Retrieved Contexts:")
            # for i in range(15):
            #     print(f"{i}: {context_data[context_data['id'] == index_edit[i]]['context'].values[0]}")

            # print("--")

            # if top_5 > 20:
            #     raise Exception("Debugging")
        elif idx_search < 10:
            top_10 += 1
        if idx_search < mrr_rank:
            mrr_score += 1 / (idx_search + 1)
    return (
        top_1 / len(question_all),
        top_5 / len(question_all),
        top_10 / len(question_all),
        mrr_score / len(question_all),
    )

In [9]:
def get_ds(question_data, context_data, aug_context_col=None, aug_context_ratio=0., context_col="context", questions_col="question", filter_zero_dist=True):
    """Function to return the questions and contexts, and their ids for use in benchmarking"""

    # Assert that aug_context_col is not None if aug_context_ratio is not 0
    assert not (aug_context_col is None and aug_context_ratio > 0), "aug_context_col cannot be None if aug_context_ratio is not 0"

    # Get the questions and contexts
    questions = question_data[questions_col].values
    question_ids = question_data["context_id"].values

    # Get the contexts
    context = context_data[context_col].values
    context_ids = context_data["id"].values

    # Get the augmented contexts
    if aug_context_col:
        aug_context = context_data[aug_context_col].values
        distances = context_data[aug_context_col.replace("th_", "dis_")].values

        if filter_zero_dist:
            aug_context = aug_context[distances > 0]
            aug_context_ids = context_data["id"].values[distances > 0]

            # Filter out the zero distances
            distances = distances[distances > 0]
        else:
            aug_context_ids = context_data["id"].values
        
        # Sample bottom augmented contexts by aug_context_ratio (closest distance)
        if aug_context_ratio > 0:
            # Get the bottom indices
            bottom_indices = np.argsort(distances)[:int(len(distances) * aug_context_ratio)]
            # Get the bottom contexts
            aug_context = aug_context[bottom_indices]
            aug_context_ids = aug_context_ids[bottom_indices]
        else:
            # Raise error if aug_context_ratio is 0 but aug_context_col is not None
            assert aug_context_ratio != 0, "aug_context_ratio cannot be 0 if aug_context_col is not None"
        
        # Augment into context
        context = np.concatenate([context, aug_context], axis=0)
        context_ids = np.concatenate([context_ids, aug_context_ids], axis=0)
    
    return question_ids, context_ids, questions, context


def benchmark_single(q_ids, c_ids, q, c, model_embed=model_embed):
    # Check if column has a distance counterpart
    question_all = encode_in_batch(model_embed, q, progress=False)
    context_all = encode_in_batch(model_embed, c, progress=False)

    question_ids = q_ids
    context_ids = c_ids

    top_1, top_5, top_10, mrr_score = evaluate(
        question_ids, question_all, context_ids, context_all, status=False
    )
    return top_1, top_5, top_10, mrr_score

In [10]:
context_data

,id,context,th_aug,th_fasttext_aug,th_llm_gec_aug,th_llm_paraphrase_aug,th_qcpg_0.2_aug,th_qcpg_0.2_llm_gec_aug,th_qcpg_0.5_aug,th_qcpg_0.5_llm_gec_aug,...,dis_llm_paraphrase_aug,dis_qcpg_0.2_aug,dis_qcpg_0.2_llm_gec_aug,dis_qcpg_0.5_aug,dis_qcpg_0.5_llm_gec_aug,dis_qcpg_0.8_aug,dis_qcpg_0.8_llm_gec_aug,dis_wordnet_aug,context_id,source
0,9ce376f4446e5b43e55d6f545c1adec7975a51d9085156...,พัทธ์ธีรา ศรุติพงศ์โภคิน (เกิด 3 ธันวาคม พ.ศ. ...,Phatthira Saruritphong Phokin (เกิด 3 ธันวาคม ...,พัทธ์ธีดาเล ศรุติพงศ์โภคิน (เกิด 3 21มกราคม พ....,Phatthira Saruritphong Phokin (เกิด 3 ธันวาคม ...,Phatthira Saruritphong Phokin ซึ่งเกิดเมื่อวัน...,Phatthira Saruuritt Phokin (เกิดเมื่อวันที่ 3 ...,Phatthira Saruutfokin (เกิดเมื่อวันที่ 3 ธันวา...,ชื่อของ Phatthira Saruuritt Phokin (เกิดในโรงเ...,Phatthira Saruutfokin (เกิดเมื่อวันที่ 3 ธันวา...,...,0.184912,0.436465,0.437679,0.501482,0.370968,0.450085,0.369090,0.022316,9ce376f4446e5b43e55d6f545c1adec7975a51d9085156...,iapp
1,54421ea8bf2c7778b1d27a9e878c31aee3361455e767f0...,คลีโอพัตราที่ 7 ฟิโลพาเธอร์ (กรีก: Κλεοπάτρα θ...,คลีโอพัตรา 7 ฟิโลพาเธอร์ (กรีก: κλεοπάτραθεάφι...,คลีโอพัตราที่ 58 Zafina ทริงเงิพาเธอร์ (กรีก :...,Cleopatra VII Philopator (กรีก: κλεοπάτραθεάφι...,Cleopatra VII Philopator หรือที่รู้จักกันในชื่...,"- 69 - 69 - โต๊ะ (Laug พ่อของเธอ A Wolf Mi, 12...",Cleopatra VII หรือที่รู้จักกันในชื่อ Cleopatra...,ดังนั้นครอบครัวกรีกคนสุดท้ายของราชวงศ์สตากา-เม...,Cleopatra VII หรือที่รู้จักกันในชื่อคลีโอพัตรา...,...,0.141594,0.292852,0.208292,0.220980,0.210469,0.274992,0.226844,0.018455,54421ea8bf2c7778b1d27a9e878c31aee3361455e767f0...,iapp
2,287fa29b965a1ba84011bb12ffafb5194984a955d851eb...,ออสเตรเลีย (อังกฤษ: Australia) หรือชื่อทางการค...,ออสเตรเลียหรือชื่อทางการคือกลุ่มออสเตรเลีย (เค...,ออสเตรเลีย6 (อังกฤษ : Australia ) บางทีชื่อทาง...,ออสเตรเลียซึ่งเป็นที่รู้จักอย่างเป็นทางการในฐา...,ออสเตรเลียซึ่งเป็นที่รู้จักอย่างเป็นทางการในฐา...,"กลุ่มออสเตรเลีย (ออสเตรเลีย), Tae-Samanthin แล...",ออสเตรเลียซึ่งเรียกว่าออสเตรเลียอย่างเป็นทางกา...,"กลุ่มออสเตรเลีย (ออสเตรเลีย), Tae-Samanthin แล...",กลุ่มออสเตรเลียหรือที่รู้จักกันในชื่อกลุ่มออสเ...,...,0.061920,0.149705,0.146146,0.117441,0.146885,0.128104,0.151757,0.024578,287fa29b965a1ba84011bb12ffafb5194984a955d851eb...,iapp
3,35adee7e78d0ae214238db68c9b524e01b1997ee7bf1ad...,วันสันติภาพไทย คือ วันที่ 16 สิงหาคม ของทุกปี ...,วันสันติภาพไทยเป็นวันที่ 16 สิงหาคมทุกปีซึ่งเป...,เมื่ออนุสนธิการ์ตูน2ภาษา เรียก วันที่ YIMSOO ส...,วันแห่งความสงบสุขของไทยมีการเฉลิมฉลองวันที่ 16...,วันสันติภาพไทยมีการเฉลิมฉลองเป็นประจำทุกปีในวั...,วันสันติภาพไทยเป็นวันที่ 16 ของ AUGUST 1945 แล...,วันแห่งความสงบสุขของไทยมีการเฉลิมฉลองในวันที่ ...,วันสันติภาพไทยเป็นวันที่ 16 ของ AUGUST 1945 แล...,วันแห่งความสงบสุขของไทยมีการเฉลิมฉลองในวันที่ ...,...,0.136226,0.256062,0.181397,0.295456,0.207665,0.227967,0.247443,0.010985,35adee7e78d0ae214238db68c9b524e01b1997ee7bf1ad...,iapp
4,e4952f44ce25aa622976eb9d382ae03cfa38d3d081f0df...,นิวซีแลนด์ (อังกฤษ: New Zealand มาวรี: Aotearo...,นิวซีแลนด์ (ภาษาอังกฤษ: นิวซีแลนด์: Aotearoa ห...,นิวซีแลนด์ (อังกฤษ : New Zealand มาวอัลแฟ : Ao...,นิวซีแลนด์ (ภาษาอังกฤษ: นิวซีแลนด์ เมารี: Aote...,นิวซีแลนด์หรือที่รู้จักกันในชื่อ Aotearoa หรือ...,นิวซีแลนด์ (Aotearoa) เป็นดินแดนที่มีเมฆสีขาวอ...,"นิวซีแลนด์ (Aotearoa, นิวซีแลนด์, ดินแดนแห่งเม...",นิวซีแลนด์ (Aotearoa: Aotearoa ประเทศฟินแลนด์ป...,"นิวซีแลนด์ (Aotearoa, นิวซีแลนด์และสาธารณรัฐอั...",...,0.092218,0.196770,0.132868,0.152546,0.152899,0.167104,0.194042,0.014894,e4952f44ce25aa622976eb9d382ae03cfa38d3d081f0df...,iapp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7838,5f74195cacdd4383213ee1ad9cc2e41497f0b92a499572...,สีดำ หมายถึง สีแห่งศอของพระศิวะที่ดื่มยาพิษเพื...,สีดำหมายถึงสีของพระอิศวรที่ดื่มเพื่อปกป้องมนุษ...,สีดำ หมายถึง สีแห่งศอของพระศิวะที่ดื่มยาพิษเพื...,สีดำเป็นสัญลักษณ์ของอะไรในบริบทของเรื่องราวของ...,สีดำเป็นสีที่เกี่ยวข้องกับพระอิศวรซึ่งเป็นการแ...,สีดำเป็นสีของพระอิศวรคนที่ดื่มเพื่อปกป้องมนุษย...,สีดำคืออะไรเมื่อเผชิญกับเรื่องราวของ SIV เก

In [11]:
# Get all unique source of contexts
context_data["source"].unique()

array(['iapp', 'thaiqa', 'xquad', 'tydiqa'], dtype=object)

In [12]:
results = []

unique_sources = list(context_data["source"].unique())

for source in tqdm(unique_sources, desc="Iterating through datasets"):
    for col in tqdm(all_augment_cols, desc="Iterating through augmentations", leave=False):
        if col is not None:
            for ratio in tqdm(range(1, 11), desc="Iterating through ratios", leave=False, total=10, miniters=0):
                ratio = ratio / 10

                # Get the data
                question_ids, context_ids, questions, context = get_ds(questions_data[questions_data["source"] == source], context_data[context_data["source"] == source], aug_context_col=col, aug_context_ratio=ratio)
                
                top_1, top_5, top_10, mrr_score = benchmark_single(question_ids, context_ids, questions, context)
                results.append({
                    "source": source,
                    "augment_col": col,
                    "augment_ratio": ratio,
                    "top_1": top_1,
                    "top_5": top_5,
                    "top_10": top_10,
                    "mrr_score": mrr_score
                })
        else:
            # Get the data
            question_ids, context_ids, questions, context = get_ds(questions_data[questions_data["source"] == source], context_data[context_data["source"] == source], aug_context_col=col, aug_context_ratio=0)
            
            top_1, top_5, top_10, mrr_score = benchmark_single(question_ids, context_ids, questions, context)
            results.append({
                "source": source,
                "augment_col": col,
                "augment_ratio": 0,
                "top_1": top_1,
                "top_5": top_5,
                "top_10": top_10,
                "mrr_score": mrr_score
            })
            # pass

results = pd.DataFrame(results)
results

Iterating through datasets:   0%|          | 0/4 [00:00<?, ?it/s]

Iterating through augmentations:   0%|          | 0/12 [00:00<?, ?it/s]

Iterating through ratios:   0%|          | 0/10 [00:00<?, ?it/s]

Iterating through ratios:   0%|          | 0/10 [00:00<?, ?it/s]

Iterating through ratios:   0%|          | 0/10 [00:00<?, ?it/s]

...

Iterating through ratios:   0%|          | 0/10 [00:00<?, ?it/s]

Iterating through ratios:   0%|          | 0/10 [00:00<?, ?it/s]

Iterating through ratios:   0%|          | 0/10 [00:00<?, ?it/s]

Iterating through ratios:   0%|          | 0/10 [00:00<?, ?it/s]

,source,augment_col,augment_ratio,top_1,top_5,top_10,mrr_score
0,iapp,None,0.0,0.455300,0.651121,0.714088,0.539562
1,iapp,th_aug,0.1,0.454470,0.646554,0.710352,0.536682
2,iapp,th_aug,0.2,0.453778,0.638666,0.705093,0.532926
3,iapp,th_aug,0.3,0.452117,0.634099,0.701218,0.529531
4,iapp,th_aug,0.4,0.447135,0.627872,0.695821,0.523743
...,...,...,...,...,...,...,...
439,tydiqa,th_wordnet_aug,0.6,0.339155,0.550741,0.646916,0.426648
440,tydiqa,th_wordnet_aug,0.7,0.335397,0.544992,0.639399,0.420789
441,tydiqa,th_wordnet_aug,0.8,0.331417,0.540349,0.633871,0.416962
442,tydiqa,th_wordnet_aug,0.9,0.325006,0.525094,0.623480,0.408083


In [13]:
results = pd.DataFrame(results)
results

,source,augment_col,augment_ratio,top_1,top_5,top_10,mrr_score
0,iapp,None,0.0,0.455300,0.651121,0.714088,0.539562
1,iapp,th_aug,0.1,0.454470,0.646554,0.710352,0.536682
2,iapp,th_aug,0.2,0.453778,0.638666,0.705093,0.532926
3,iapp,th_aug,0.3,0.452117,0.634099,0.701218,0.529531
4,iapp,th_aug,0.4,0.447135,0.627872,0.695821,0.523743
...,...,...,...,...,...,...,...
439,tydiqa,th_wordnet_aug,0.6,0.339155,0.550741,0.646916,0.426648
440,tydiqa,th_wordnet_aug,0.7,0.335397,0.544992,0.639399,0.420789
441,tydiqa,th_wordnet_aug,0.8,0.331417,0.540349,0.633871,0.416962
442,tydiqa,th_wordnet_aug,0.9,0.325006,0.525094,0.623480,0.408083


In [14]:
all_augment_cols

[None,
 'th_aug',
 'th_fasttext_aug',
 'th_llm_gec_aug',
 'th_llm_paraphrase_aug',
 'th_qcpg_0.2_aug',
 'th_qcpg_0.2_llm_gec_aug',
 'th_qcpg_0.5_aug',
 'th_qcpg_0.5_llm_gec_aug',
 'th_qcpg_0.8_aug',
 'th_qcpg_0.8_llm_gec_aug',
 'th_wordnet_aug']

In [ ]:
results.to_csv("../results/benchmark_reqa.csv", index=False)